Reference: Piasecka, E. et al. Natural variation in the parameters of innate immune cells is preferentially driven by genetic factors resource. Nat. Immunol. 19, 302-314 (2018). Piasecka, B. et al. Distinctive roles of age, sex, and genetics in shaping transcriptional variation of human immune responses to microbial challenges. Proc. Natl. Acad. Sci. 115, E488-E497 (2018). и http://www.milieuinterieur.fr/

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sc
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import warnings
import numpy as np
import itertools
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import scale, OneHotEncoder
from sklearn.metrics import explained_variance_score, mean_squared_log_error, r2_score
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")
%matplotlib inline

Данный датасет представляет собой рнк-секвенирование иммуного ответа пациентов на различные стимулы  (Escherichia coli, BCG, Staphylococcus aureus, SEB, Candida albicans and Influenza virus). В данном исследование были секвенированы 560 генов. Основная задача - оценить, насколько может быть вариабелен иммунный ответ в рамках популяции (пусть даже выборка тут небольшая)

Первые колонки, которые в своем названии содержат MFI - это не гены, это усредненный показатель наличиние антитела Н в популяции клеток К. Например, CD38_MFI_in_Bcells  показывает уровень CD38 в B-лимфоцитах. Данные показатели и будут показывать "уровень" имунного ответа. Чем больше этот показатель - тем больше клеток такого типа мы наблюдаем. 
В данной работе мы сосредоточимся на следующийх генах - IFNA2 (возраст) IFNG(возраст),  MAPK14, GATA3

In [ ]:
dt = pd.read_csv('dataset_merged.txt',sep='\t')
dt.head()

Проведем небольшую разминку - построим графики, посмотрим на распределения, проведем тесты, посчитаем статистики. 

У нас есть дополнительная информация. Самые интересные из колонок:
 - Age
 - PhysicalActivity
 - Sex
 - MetabolicScore
 - HoursOfSleep
 - UsesCannabis
 - Smoking
 - BMI

 Посмотрим на распределение возрастов и пола

In [ ]:
metadata_columns = ['id','Age','PhysicalActivity','Sex','MetabolicScore','BMI','HoursOfSleep','UsesCannabis',\
                    'Smoking','Employed','Education','DustExposure']
ids_description  = dt.drop_duplicates('id').drop(['stimulus'],axis=1)[metadata_columns]
ids_description.head()

In [ ]:
sns.pairplot(ids_description[['Age','PhysicalActivity','MetabolicScore','BMI','HoursOfSleep']]);
plt.show()
sns.catplot(x="Sex", y="Age", hue="Smoking",data=ids_description,kind="violin");

В принципе, данные довольно сбалансированные. Не удивительно, ведь авторы исследования так и пытались сделать. 
Теперь начнем смотреть на экспрессии генов и их связь с возрастом\полом\статусом.

3.1 Сравнение экспресии IFNG у мужчин и женщин

3.1.1 Рассмотрим распределение экспресси IFNG у женщин (со стимулом - S.aureus). На что похоже это распределение? Как это проверить? 

In [ ]:
mask = (dt['Sex']=='Female') & (dt['stimulus']=='S.aureus')
sns.distplot(dt.loc[mask,'IFNG']);

Какое удивительное нормальные распределение! Давайте проверим, что оно нормальное

In [ ]:
k2, p = sc.stats.shapiro(dt.loc[mask,'IFNG'].values)
print('p value {}'.format(p))
sc.stats.probplot(dt.loc[mask,'IFNG'].values,plot=plt);

 Оценим параметры этого распределения.

In [ ]:
d_var = np.var(dt.loc[mask,'IFNG'].values)
d_mean = np.mean(dt.loc[mask,'IFNG'].values)
print('mean {}, var {}'.format(d_mean, d_var))

Задача 1. Сделать аналогичные шаги для мужчин (со стимулом - S.aureus)

In [ ]:
#Task 1 solution
ax1 = plt.subplot(221)
mask = (dt['Sex']=='Male') & (dt['stimulus']=='S.aureus')
sns.distplot(dt.loc[mask,'IFNG'],ax=ax1);
plt.show();
k2, p = sc.stats.shapiro(dt.loc[mask,'IFNG'].values)
print('p value {}'.format(p))
d_var = np.var(dt.loc[mask,'IFNG'].values)
d_mean = np.mean(dt.loc[mask,'IFNG'].values)
#Funny fact - outliers ruin normaltest?
ax2 = plt.subplot(223)
sc.stats.probplot(dt.loc[mask,'IFNG'].values, plot=plt);
print('mean {}, var {}'.format(d_mean, d_var))

Задача 2. Давайте используем бустреп чтобы оценить доверительный интервал для среднего уровня экспресии гена
GATA3 у мужчин и женщин. Используем 1000 итераций, будем семплить по 100 точек. Давайте условимся, что в этот раз работаем со стимулом E.coli

In [ ]:
#Task 2 solution
stats = {'Male':list(),'Female':list()}
n_iterations = 1000
n_size = 100
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), sc.stats.sem(a)
    h = se * sc.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h
f, ax = plt.subplots(1, 2, figsize=(16, 8))
for idx, sex in enumerate(['Male', 'Female']):
    print('Boostrap for {}'.format(sex))
    mask = (dt['Sex']==sex) & (dt['stimulus']=='E.coli')
    values_expression = dt.loc[mask,'IFNG'].values
    for i in range(n_iterations):
    # prepare train and test sets
        sample = np.random.choice(values_expression, size=n_size, replace=True)
        stats[sex].append(np.mean(sample))
    ax[idx].hist(stats[sex]);
    ax[idx].set_title(sex)
    m, l, h = mean_confidence_interval(stats[sex])
    alpha = 0.95
    print('for {}, {} confidence interval {} and {}'.format(sex, alpha*100, l, h))

Выглядит так, будто экспрессия и правда различна. Давайте посмотрим на совместное распределение. В этом нам поможет violin plot

In [ ]:
dt_subset = dt.loc[dt['stimulus']=='E.coli',:]
sns.catplot(x="Sex", y="IFNG", kind="violin", data=dt_subset);

Сложно сказать однозначно... Что тут можно сделать? 

Задача 3. Проверим, является ли различие в средней экспресии IFNG у мужчин и женщин при E.coli статистически значимым.
Каким тестом тут лучше воспользоваться и почему?  (при уровне значимости $\alpha  = 0.05$)

In [ ]:
#Task 3 solution
_, pval = sc.stats.ttest_ind(dt_subset.loc[(dt_subset['Sex']=='Male'),'IFNG'],
                             dt_subset.loc[(dt_subset['Sex']=='Female'),'IFNG'])
print('pvalue is {}'.format(pval))
if pval < 0.05 :
    print('Yeap')
else:
    print('Nope')

Задача 4. А интересно, у скольки еще генов различие в средней экспресии у мужчин и женщин при E.coli статистически значимым (при уровне значимости $\alpha  = 0.05$). Не будем усложнять себе жизнь и воспользуемся поправкой Бонферрони

In [ ]:
#Task 4 solution
from statsmodels.sandbox.stats.multicomp import multipletests
gene_lists = dt_subset.columns.values[210:-1]
pvals_list = []
for gene in gene_lists:
    _, pval = sc.stats.ttest_ind(dt_subset.loc[dt_subset['Sex']=='Male', gene], dt_subset.loc[dt_subset['Sex']=='Female', gene])
    pvals_list.append(pval)
p_adjusted = multipletests(pvals_list, method='bonferroni')[1]
passed_genes = sum(p_adjusted<0.05)
print('{} genes are different between males and females.'.format(passed_genes))

Отлично, но вернемся к IFNG. Добавим к рассмотрниею следующие параметры - Age, PhysicalActivity, BMI, Smoking, Education, LivesWithPartner. Как можно заметить, некоторые из них категориальные, некоторые - числовые. Тем интересней!

Задача 5. Рассмотрите уровень экспресии IFNG (при E.coli)  в зависимости от каждого из вышеназванных параметров. 
Какой вывод можно сделать? А если дополнительно разбить с учетом пола? 

In [ ]:
#Task 5 solution
fontsize=18
subset_data = dt.loc[dt['stimulus']=='E.coli', ['Age','PhysicalActivity','BMI','Smoking','Sex',
                                                'LivesWithPartner','Education', 'IFNG']]
# Plot sepal with as a function of sepal_length across days
numeric_columns  = ['Age','PhysicalActivity','BMI']
cat_columns  = ['Smoking','LivesWithPartner','Education']
f, ax = plt.subplots(1, len(numeric_columns), figsize=(24, 8))
for idx, numberic_col in enumerate(numeric_columns):
    sns.scatterplot(x=numberic_col, y="IFNG", data=subset_data, ax=ax[idx])
plt.show()

f, ax = plt.subplots(1, len(cat_columns), figsize=(24, 8))    
for idx, cat_col in enumerate(cat_columns):
    sns.boxplot(x=cat_col, y="IFNG", palette=["m", "g"],
            data=subset_data, ax=ax[idx])
plt.show()    

f, ax = plt.subplots(1, len(numeric_columns), figsize=(24, 8))
for idx, numberic_col in enumerate(numeric_columns):
    g = sns.scatterplot(x=numberic_col, y="IFNG", hue="Sex", data=subset_data, ax=ax[idx])
plt.show()

f, ax = plt.subplots(1, len(cat_columns), figsize=(24, 8))      
for idx, cat_col in enumerate(cat_columns):
    g = sns.boxplot(x=cat_col, y="IFNG",
            hue="Sex", palette=["m", "g"],
            data=subset_data, ax=ax[idx])
plt.show()

Задача 6. Для того, чтобы рассматривать экспрессию между несколькими группами, нам понадобится знание диспресионного анализа. 
Проверьте, можем ли мы утверждать, что уровень экспресии IFNG не зависит от курения (без разбиения по полу) (используейте пакет $statsmodels$). Дополнительно проведите Tukey hsd чтобы оценить разницу между группами

In [ ]:
#Task 6 solution
f_value, p_value = sc.stats.f_oneway(subset_data.loc[subset_data['Smoking']=='Never','IFNG'].values,
                                     subset_data.loc[subset_data['Smoking']=='Active','IFNG'].values, 
                                     subset_data.loc[subset_data['Smoking']=='Ex','IFNG'].values)
print('One way anove p value {}'.format(p_value))
print(pairwise_tukeyhsd(subset_data['IFNG'], subset_data['Smoking']))

В последующих шагах по постараемся воспроизвести основные результаты статьи  Piasecka, E. et al. Natural variation in the parameters of innate immune cells is preferentially driven by genetic factors resource. Nat. Immunol. 19, 302-314 (2018), опубликованной в Nature Genetics. Далее мы работаем с стимулом NS, то есть с изначальным иммуным ответом.

In [ ]:
dt_ns  = dt.loc[dt['stimulus']=='NS',:]

Большое Задание 1. Рассмотрите зависимости между предложенными негентическими переменными в этом датасете. Для это сделаете попарные линейные модели и в качестве зависимости возьмите $R^2$. При работе с категориальными переменными, используйте OHE стратегию и используйте их только как предикторы. (Хотя в случае бинарного лейбла можно сделать логрегрессию)

In [ ]:
nongenetic_columns = ['Age', 'OwnsHouse',
                      'PhysicalActivity', 'Sex', 'LivesWithPartner', 'LivesWithKids',
                      'BornInCity', 'Inbreeding', 'BMI', 'CMVPositiveSerology', 'FluIgG',
                      'MetabolicScore', 'LowAppetite', 'TroubleConcentrating',
                      'TroubleSleeping', 'HoursOfSleep', 'Listless', 'UsesCannabis',
                      'RecentPersonalCrisis', 'Smoking', 'Employed', 'Education',
                      'DustExposure', 'Income',
                      'DepressionScore', 'HeartRate',
                      'Temperature']

In [ ]:
#Big task 1 solution
#make combinations of feature-outcome
combinations =  list(itertools.permutations(nongenetic_columns, 2))
#make zero matrix for results
dep_matrix = np.zeros((len(nongenetic_columns),len(nongenetic_columns)))
#self-self is 1
np.fill_diagonal(dep_matrix, 1)
idx_columns = dict(zip(nongenetic_columns, np.arange(len(nongenetic_columns))))
#identify which of the columns as categorical
cat_features = []
for col_ in nongenetic_columns:
    vals = dt_ns[col_].value_counts()
    n_features = len(set(vals.index))
    top_5_features = 100*vals.values[:5].sum()/vals.sum()
    print('Feature {}, total features {}, top 5 account for {}'.format(col_,n_features,top_5_features))
    if n_features<=5 or top_5_features>=96:
        cat_features.append(col_)
        print('Can use as category\n')
print('='*50)
print(' '*20+'Making linear regression'+' '*20)
print('='*50)
#columns - predictors (first), rows = dependent (second)
for pair in combinations:
    print(pair)
    if pair[1] in cat_features:
        print('{} is categorical, skip as dependent'.format(pair[1]))
        continue
    if pair[0] in cat_features:
        print('{} is categorical, making dummy variables'.format(pair[0]))
        X = pd.get_dummies(dt_ns[pair[0]].astype(object))
        y = dt_ns[pair[1]]
        reg = LinearRegression().fit(X, y)
        r_squared = reg.score(X,y)
        dep_matrix[idx_columns[pair[1]],idx_columns[pair[0]]] = r_squared
    else:
        print('{} is numeric'.format(pair[0]))
        X = dt_ns[pair[0]].values.reshape(-1, 1)
        y = dt_ns[pair[1]].values
        reg = LinearRegression().fit(X, y)
        r_squared = reg.score(X,y)
        dep_matrix[idx_columns[pair[1]],idx_columns[pair[0]]] = r_squared

f, ax = plt.subplots(1, 1, figsize=(16, 16))
ax.imshow(dep_matrix, cmap="YlGn");
ax.set_xticks(np.arange(len(nongenetic_columns)));
ax.set_yticks(np.arange(len(nongenetic_columns)));
# ... and label them with the respective list entries
ax.set_xticklabels(nongenetic_columns);
ax.set_yticklabels(nongenetic_columns);
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor");
for i in range(len(nongenetic_columns)):
    for j in range(len(nongenetic_columns)):
        text = ax.text(j, i, '%.2f' % dep_matrix[i, j],
                       ha="center", va="center", color="k")

ax.set_title("Dependency matrix")

Большое задание 1.1. Теперь мы посмотрим на корреляцию между различными MFI. Это нужно, чтобы дальше мы взяли в работу только нескореллированные признаки

In [ ]:
MFI_cols = [x for x in dt_ns.columns.values if 'MFI_' in x]

In [ ]:
#Big task 1.1 solution 
corr_matrix = dt_ns[MFI_cols].corr()
corr_matrix.values[np.abs(corr_matrix.values)<0.3] = 0
f, ax = plt.subplots(1, 1, figsize=(16, 16))
ax.imshow(corr_matrix.values, cmap="YlGn");
ax.set_xticks(np.arange(len(MFI_cols)));
ax.set_yticks(np.arange(len(MFI_cols)));
# ... and label them with the respective list entries
ax.set_xticklabels(MFI_cols);
ax.set_yticklabels(MFI_cols);
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor");
for i in range(len(MFI_cols)):
    for j in range(len(MFI_cols)):
        if corr_matrix.values[i, j]>=0.9 and i!=j:
            text = ax.text(j, i, '%.2f' % corr_matrix.values[i, j],
                           ha="center", va="center", color="w")

ax.set_title("MFI correlation matrix");
#Select feature if only all correlation are less than 0.6  - quick and dirty 
noncorrelated_features = corr_matrix.apply(lambda x: all(x[x!=1]<0.8))
noncorrelated_features = noncorrelated_features.index[noncorrelated_features==True].tolist()

Большое задание 2. Теперь сделаем регресионные модели для каждого из этих показателей.

In [ ]:
factors = ['Age', 'Sex', 'BMI', 'CMVPositiveSerology', 'MetabolicScore', 'Smoking']
dt_ns_regres = dt_ns[factors+noncorrelated_features]

In [ ]:
#scale features
#Big task 2 solutions
cat_features = ['Sex','CMVPositiveSerology','Smoking']
numeric_features = ['Age','BMI','MetabolicScore']
dt_ns_regres[numeric_features] = scale(dt_ns_regres[numeric_features])
for col_ in cat_features:
    dt_ns_regres[col_] = dt_ns_regres[col_].astype(object)
X_cat = pd.get_dummies(dt_ns_regres[cat_features])
X_cat = X_cat.drop(['Sex_Female','CMVPositiveSerology_No','Smoking_Never'],axis=1)
X = pd.concat([X_cat, dt_ns_regres[numeric_features]],axis=1)
Y = dt_ns_regres[noncorrelated_features]
regress_features = X.columns.values.tolist()
del dt_ns_regres
for MFI_ in noncorrelated_features:
    print('processing {}'.format(MFI_))
    data = pd.merge(X,Y[[MFI_]], left_index=True, right_index=True)
    data = data.dropna(axis=0)
    #data = data.fillna(0)
    print('Columns after dropping NA: {}'.format(data.shape[0]))
    X_train, X_test, y_train, y_test = train_test_split(data[regress_features], data[MFI_], test_size=0.2, random_state=42)
    reg = LinearRegression()
    reg.fit(X_train, y_train)
    preds = reg.predict(X_test)
    rmsle_ = mean_squared_log_error(y_test, preds)
    print('RMSE : {}'.format(rmsle_))